In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import importlib
import json
import os
import mne

import combined_analysis_bachelor.code.read_in_emg_acc as read_in
import combined_analysis_bachelor.code.movement_detection_functions as move_functions
import combined_analysis_bachelor.code.create_processed_files as process
import combined_analysis_bachelor.code.functions_for_pipeline as funcs
import my_utils.get_sub_dir as get_sub_dir
from my_utils.get_sub_dir import get_sub_folder_dir
import my_utils.find_paths as find_path

In [18]:
SUB = "91"

out_dir = get_sub_dir.get_sub_folder_dir(SUB, "onset_data")
out_events_dir = f"{out_dir}/sub-{SUB}_events.json"
out_threshs_dir = f"{out_dir}/sub-{SUB}_thresholds.json"
out_labels_dir = f"{out_dir}/sub-{SUB}_behavioral_labels.json"

with open(out_events_dir, "r", encoding="utf-8") as f:
    events_result = json.load(f)
with open(out_threshs_dir, "r", encoding="utf-8") as f:
    thresholds_result = json.load(f)
with open(out_labels_dir, "r", encoding="utf-8") as f:
    labels_result = json.load(f)  


# === 2) Auswahl: Datei, Limb, Channels ===
file_id = "sub-91_EmgAcc_setupB_RestMockDys_processed"
limb    = "legL"
channels = ["tibialisAnterior_L_tkeo","SVM_R"] #"brachioradialis_L_tkeo", ]

project folder found: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG
C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG\data\onset_data


In [31]:
def get_events(events_dict, file_id, limb, channel):
    ch = events_dict.get(file_id, {}).get(limb, {}).get(channel, {})
    on_s  = ch.get("onsets_seconds", [])   # Liste[float]
    off_s = ch.get("offsets_seconds", [])  # Liste[float]
    return on_s, off_s

def get_threshold(thr_dict, file_id, limb, channel):
    return thr_dict.get(file_id, {}).get(limb, {}).get(channel, {}).get("threshold", None)

# for plotting # 
def shade_intervals(ax, on_list, off_list, alpha=0.15):
    for s, e in zip(on_list, off_list):
        if e is not None and s is not None:
            ax.axvspan(s, e, color="slategray",alpha=alpha)

def draw_threshold(ax, thr):
    if thr is not None:
        ax.axhline(thr, linestyle="--", linewidth=1, label="thresh")

In [400]:
chan_data = {}
for ch in channels:
    on_s, off_s = get_events(events_result, file_id, limb, ch)
    thr         = get_threshold(thresholds_result, file_id, limb, ch)
    chan_data[ch] = {"on": on_s, "off": off_s, "thr": thr}

In [401]:
chan_data

{'tibialisAnterior_L_tkeo': {'on': [0.0,
   0.56,
   6.41,
   8.13,
   8.37,
   83.95,
   86.61,
   87.32,
   88.38,
   90.28,
   91.01,
   91.45,
   92.16,
   92.51,
   94.03,
   94.52,
   95.96,
   96.86,
   97.05,
   97.64,
   97.9,
   98.15,
   99.61,
   100.09,
   100.59,
   100.81,
   101.55,
   102.5,
   103.46,
   104.65,
   105.56,
   106.68,
   107.51,
   109.08,
   110.14,
   112.07,
   113.06,
   114.52,
   115.52,
   115.73,
   116.77,
   117.35,
   118.34,
   118.79,
   121.64,
   122.38,
   125.26,
   126.02,
   131.6,
   132.89,
   133.95,
   136.77,
   138.5,
   140.85,
   141.28,
   203.04,
   204.01,
   205.33,
   206.28,
   206.62,
   207.79,
   208.21,
   208.45,
   209.59,
   212.07,
   213.22,
   213.74,
   217.57,
   218.53,
   219.25,
   219.88,
   221.25,
   221.7,
   222.95,
   223.51,
   223.98,
   225.72,
   226.37,
   227.04,
   227.3,
   233.41,
   234.35,
   235.03,
   237.17,
   237.61,
   238.04,
   238.31,
   239.07,
   239.38,
   239.86,
   242.78,
 

In [402]:
chan_data["SVM_R"] # irgendwas ist mit bein komisch, falsch gethreshed?? OPTIONAL!!!

{'on': [3.83, 83.49, 142.79, 202.67, 322.05, 383.66, 384.01],
 'off': [4.75, 142.66, 143.63, 263.99, 383.6, 383.79, 385.04],
 'thr': 0.0072740675375744844}

In [19]:
# hier filtered data kriegen:
proc_dir = get_sub_dir.get_sub_folder_dir(SUB, "processed_data")
move_dir = os.path.join(proc_dir, "sub-91_EmgAcc_setupB_Move_processed.h5")
move = pd.DataFrame(pd.read_hdf(move_dir, key="data"))

mock_dir = os.path.join(proc_dir, "sub-91_EmgAcc_setupB_RestMockDys_processed.h5")
mock = pd.DataFrame(pd.read_hdf(mock_dir, key="data"))

rest_dir = os.path.join(proc_dir, "sub-91_EmgAcc_setupB_Rest_processed.h5")
rest = pd.DataFrame(pd.read_hdf(rest_dir, key="data"))

df_move = funcs.add_tkeo(move, ["brachioradialis_L", "deltoideus_L"], 20)
df_mock = funcs.add_tkeo(mock, ["tibialisAnterior_L"], 20)
df_rest = funcs.add_tkeo(rest, ["brachioradialis_R", "deltoideus_R"], 20)

project folder found: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG
C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG\data\processed_data


In [72]:
fig_path = find_path.get_onedrive_path(folder="figures")
mock_leg_path = f"{fig_path}/mix_EMG_ACC_figures/sub-91_B_RestMockDys_legL_data_withActive.svg"

move_armL_path = f"{fig_path}/mix_EMG_ACC_figures/sub-91_B_Move_armL_data_withActive.svg"

rest_armL_path = f"{fig_path}/mix_EMG_ACC_figures/sub-91_B_Rest_armL_data_withActive.svg"

project folder found: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG


In [54]:
# arme!
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12, 7), sharex=True)
x_min, x_max = 15,95 

# ---- Subplot 1: brachioradialis_L_tkeo ----
ax = axes[0]
ax.plot(df_move["Sync_Time (s)"], df_move["brachioradialis_L"], color="lightskyblue", label="brachioradialis")
shade_intervals(ax, chan_data["brachioradialis_L_tkeo"]["on"], chan_data["brachioradialis_L_tkeo"]["off"])
#draw_threshold(ax, chan_data["brachioradialis_L_tkeo"]["thr"])
ax.set_facecolor("white")
ax.set_xlim(x_min, x_max)
#ax.set_ylim(0, 10000)
ax.set_title("brachioradialis_L")
ax.grid(False)

# ---- Subplot 2: deltoideus_L_tkeo ----
ax = axes[1]
ax.plot(df_move["Sync_Time (s)"], df_move["deltoideus_L"], color="cornflowerblue", label="deltoideus")
shade_intervals(ax, chan_data["deltoideus_L_tkeo"]["on"], chan_data["deltoideus_L_tkeo"]["off"])
#draw_threshold(ax, chan_data["deltoideus_L_tkeo"]["thr"])
ax.set_facecolor("white")
ax.set_xlim(x_min, x_max)
#ax.set_ylim(0, 8000)
ax.set_title("deltoideus_L")
ax.grid(False)

# ---- Subplot 3: SVM_L ----
ax = axes[2]
ax.plot(df_move["Sync_Time (s)"], df_move["SVM_L"], color="steelblue", label="ACC SVM")
shade_intervals(ax, chan_data["SVM_L"]["on"], chan_data["SVM_L"]["off"])
#draw_threshold(ax, chan_data["SVM_L"]["thr"])
ax.set_facecolor("white")
ax.set_xlim(x_min, x_max)
#ax.set_ylim(0, 0.1)
ax.set_title("SVM_L")
ax.set_xlabel("Time (s)")
ax.grid(False)
ax.tick_params(bottom=True)

#ax.spines[['top', 'left', "bottom"]].set_visible(False)
sp = ax.spines["bottom"]
sp.set_linewidth(0.8)
sp.set_color("black")
sp.set_zorder(10)
#ax.set_xticks([20,30,40,50,60,70,80,90])

plt.tight_layout()
#plt.show()
fig.savefig(move_armL_path, dpi=300)

In [396]:
# arme Rest
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12, 7), sharex=True)
x_min, x_max = 15,300 


# ---- Subplot 1: brachioradialis_L_tkeo ----
ax = axes[0]
ax.plot(df_rest["Sync_Time (s)"], df_rest["brachioradialis_L"], color="lightskyblue", label="brachioradialis")
shade_intervals(ax, chan_data["brachioradialis_L_tkeo"]["on"], chan_data["brachioradialis_L_tkeo"]["off"])
#draw_threshold(ax, chan_data["brachioradialis_L_tkeo"]["thr"])
ax.set_facecolor("white")
ax.set_xlim(x_min, x_max)
#ax.set_ylim(0, 10000)
ax.set_title("brachioradialis_L")
ax.grid(False)

# ---- Subplot 2: deltoideus_L_tkeo ----
ax = axes[1]
ax.plot(df_rest["Sync_Time (s)"], df_rest["deltoideus_L"], color="cornflowerblue", label="deltoideus")
shade_intervals(ax, chan_data["deltoideus_L_tkeo"]["on"], chan_data["deltoideus_L_tkeo"]["off"])
#draw_threshold(ax, chan_data["deltoideus_L_tkeo"]["thr"])
ax.set_facecolor("white")
ax.set_xlim(x_min, x_max)
#ax.set_ylim(0, 8000)
ax.set_title("deltoideus_L")
ax.grid(False)

# ---- Subplot 3: SVM_L ----
ax = axes[2]
ax.plot(df_rest["Sync_Time (s)"], df_rest["SVM_L"], color="steelblue", label="ACC SVM")
shade_intervals(ax, chan_data["SVM_L"]["on"], chan_data["SVM_L"]["off"])
#draw_threshold(ax, chan_data["SVM_L"]["thr"])
ax.set_facecolor("white")
ax.set_xlim(x_min, x_max)
#ax.set_ylim(0, 0.1)
ax.set_title("SVM_L")
ax.set_xlabel("Time (s)")
ax.grid(False)
ax.tick_params(bottom=True)

#ax.spines[['top', 'left', "bottom"]].set_visible(False)
sp = ax.spines["bottom"]
sp.set_linewidth(0.8)
sp.set_color("black")
sp.set_zorder(10)
#ax.set_xticks([20,30,40,50,60,70,80,90])

plt.tight_layout()
#plt.show()
fig.savefig(rest_armL_path, dpi=300)

KeyError: 'brachioradialis_L_tkeo'

In [413]:
# für bein!

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(12, 7), sharex=True)
#plt.suptitle("B_RestMockDys")
x_min, x_max = 50, 300


def shade_intervals(ax, on_list, off_list, alpha=0.15):
    for s, e in zip(on_list, off_list):
        if e is not None and s is not None:
            ax.axvspan(s, e, alpha=alpha, color="slategray", zorder=0)

def draw_threshold(ax, thr):
    if thr is not None:
        ax.axhline(thr, linestyle="--", linewidth=1, label="threshold")

# ---- Subplot 1: tibialis Anterior ----
ax = axes[0]
ax.plot(df_mock["Sync_Time (s)"], df_mock["tibialisAnterior_L"], color="slateblue", label="tibialisAnterior")
shade_intervals(ax, chan_data["tibialisAnterior_L_tkeo"]["on"], chan_data["tibialisAnterior_L_tkeo"]["off"])
#draw_threshold(ax, chan_data["tibialisAnterior_L_tkeo"]["thr"])
ax.set_facecolor("white")
ax.set_xlim(x_min, x_max)
#ax.set_ylim(0, 10000)
ax.set_title("tibialisAnterior_L")
sp = ax.spines["left"]
sp.set_linewidth(0.8)
sp.set_color("black")
sp.set_zorder(10)
ax.grid(False)
ax.legend(frameon=False, loc="upper left")

# ---- Subplot 2: SVM_ leg----
ax = axes[1]
ax.plot(df_mock["Sync_Time (s)"], df_mock["SVM_R"], color="darkblue", label="SVM leg")
shade_intervals(ax, chan_data["SVM_R"]["on"], chan_data["SVM_R"]["off"])
#draw_threshold(ax, chan_data["SVM_R"]["thr"])
ax.set_facecolor("white")
ax.set_xlim(x_min, x_max)
#ax.set_ylim(0, 0.1)
ax.set_title("SVM leg")
ax.set_xlabel("Time [s]")
ax.grid(False)
for side in (["bottom", "left"]):
    sp = ax.spines[side]
    sp.set_linewidth(0.8)
    sp.set_color("black")
    sp.set_zorder(10)

for i, ax in enumerate(axes):
    # alle Spines erstmal ausschalten/setzen wie gewünscht
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # untere und linke Spine sichtbar, dünn und oben in der Zeichenreihenfolge
    for side in ['bottom', 'left']:
        sp = ax.spines[side]
        sp.set_visible(True)
        sp.set_linewidth(0.8)     # hier dünner/dicker einstellen
        sp.set_color('black')
        sp.set_zorder(10)

    # bei geteilten x-Achsen: Ticks unten aktiv lassen (Labels nur im letzten)
    ax.tick_params(bottom=True)
    if i < len(axes) - 1:
        ax.tick_params(labelbottom=False)
    
    
ax.legend(frameon=False, loc="upper left")
plt.tight_layout()
#plt.show()
fig.savefig(mock_leg_path, dpi=300)

#### now do behavioral labels plot #####

In [365]:
saving_path = find_path.get_onedrive_path(folder="figures")
saving_path = f"{saving_path}/mix_EMG_ACC_figures/sub-91_B_move_armL_behavioral_plot_legend.svg"

project folder found: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG


In [103]:
behavioral_B_move_armL = labels_result[file_id][limb]
brachio_pairs = move_functions.pairs_to_list(chan_data["brachioradialis_L_tkeo"]["on"], chan_data["brachioradialis_L_tkeo"]["off"])
delt_pairs = move_functions.pairs_to_list(chan_data["deltoideus_L_tkeo"]["on"], chan_data["deltoideus_L_tkeo"]["off"])
acc_pairs = move_functions.pairs_to_list(chan_data["SVM_L"]["on"], chan_data["SVM_L"]["off"])

In [421]:
importlib.reload(funcs)

<module 'combined_analysis_bachelor.code.functions_for_pipeline' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\functions_for_pipeline.py'>

In [105]:
funcs.plot_with_behavior(df_move, behavioral_B_move_armL, delt_pairs, brachio_pairs, acc_pairs, save=False, save_path=saving_path, dpi=300)

In [118]:
funcs.plot_with_behavior_two(df_move, behavioral_B_move_armL, delt_pairs, brachio_pairs, acc_pairs, save=False, save_path=saving_path, dpi=300)

### same for leg ###

In [428]:
saving_path = find_path.get_onedrive_path(folder="figures")
saving_path = f"{saving_path}/mix_EMG_ACC_figures/sub-91_B_RestMockDys_legL_behavioral_plot.svg"

project folder found: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG


In [429]:
importlib.reload(funcs)

<module 'combined_analysis_bachelor.code.functions_for_pipeline' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\functions_for_pipeline.py'>

In [430]:
behavioral_B_RMD_legL = labels_result[file_id][limb]
tib_pairs = move_functions.pairs_to_list(chan_data["tibialisAnterior_L_tkeo"]["on"], chan_data["tibialisAnterior_L_tkeo"]["off"])
acc_leg_pairs = move_functions.pairs_to_list(chan_data["SVM_R"]["on"], chan_data["SVM_R"]["off"])

In [431]:
funcs.plot_with_behavior_leg(df_mock, behavioral_B_RMD_legL, tib_pairs, acc_leg_pairs, save=True, save_path=saving_path, dpi=300)

### MEG plot! ###

In [159]:
# überlegen welches recording ich nehme

# MEG einlesen

# plot überlegen

# nur MEG und dadrunter gelegt sind die behavioral farben? oder so wie bei Neumann? also MEG und dann dadrunter nur in der linie das emg? 

In [14]:
filepath = find_path.get_onedrive_path("onedrive_charite", "processed_data")
path_to_B_move_OPM = os.path.join(filepath, "sub-91", "OPM_MEG", "sub-91_OPM-MEG_setupB_Move_processed.fif")
print(path_to_B_move_OPM)

project folder found: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG
C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG\data\processed_data\sub-91\OPM_MEG\sub-91_OPM-MEG_setupB_Move_processed.fif


In [15]:
processed_OPM = mne.io.read_raw_fif(path_to_B_move_OPM, preload=True, verbose=False)

C:\Users\User\AppData\Local\Temp\ipykernel_11488\3439143875.py:1: RuntimeWarning: This filename (C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG\data\processed_data\sub-91\OPM_MEG\sub-91_OPM-MEG_setupB_Move_processed.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  processed_OPM = mne.io.read_raw_fif(path_to_B_move_OPM, preload=True, verbose=False)


In [20]:
# EMG ACC data
B_move_dir = os.path.join(proc_dir, "sub-91_EmgAcc_setupB_move_processed.h5")

df_move = pd.DataFrame(pd.read_hdf(B_move_dir, key="data"))

In [21]:
print(len(df_move["Sync_Time (s)"]))
print(len(processed_OPM.times)/2000)

193260
193.26


In [22]:
data_shape = processed_OPM.get_data().shape
print(f"Data shape: {data_shape}")
print(f"Duration: {processed_OPM.times[-1]:.3f} seconds")

Data shape: (161, 386520)
Duration: 193.260 seconds


In [23]:
data_shape_emg = df_move.shape
print(data_shape_emg)

(193260, 10)


In [24]:
proc_data_opm = processed_OPM.get_data()

In [25]:
## needed functions from federico to get channel idx etc ##

def load_subject_config(subject_id: str, config_dir: str = '../configs', version: str = None) -> dict[str, any]:
    """
    Load configuration file for a specific subject and version.
    """
    config_file = os.path.join("C:/Users/User/Downloads", f'{subject_id}_config.json')
    
    if not os.path.exists(config_file):
        raise FileNotFoundError(f"Configuration file not found: {config_file}")
    
    try:
        with open(config_file, 'r') as f:
            config = json.load(f)
    except json.JSONDecodeError as e:
        raise json.JSONDecodeError(f"Invalid JSON in config file {config_file}: {e}")
    
    # If version is specified, select that version
    if version is not None:
        if version not in config:
            raise KeyError(f"Version '{version}' not found in config file {config_file}")
        config = config[version]
    
    return config


def load_and_display_config(subject_id: str, config_dir: str = '../configs', version: str = None, verbose: bool = True) -> dict[str, any]:
    """
    Convenience function to load configuration and display available options.
    """
    config = load_subject_config(subject_id, config_dir, version)
    #display_available_options(config, verbose)
    return config

In [26]:
SUBJECT_ID = "sub-91"
CONFIG_VERSION = "Version-1"

config = load_and_display_config(SUBJECT_ID, version=CONFIG_VERSION)

In [27]:
# extracting data from desired channels --> here: from right hemisphere # 
# === info === #
# F4 = frontal lobe-ish
# C4 = motor cortex-ish?
# P4 = pariteal-ish

f4_channel_name = config["meg_channels"]["F4"]
f4_channel_index = processed_OPM.ch_names.index(f4_channel_name)
f4_preprocessed_data = processed_OPM.get_data()[f4_channel_index]

c4_channel_name = config["meg_channels"]["C4"]
c4_channel_index = processed_OPM.ch_names.index(c4_channel_name)
c4_preprocessed_data = processed_OPM.get_data()[c4_channel_index]

p4_channel_name = config["meg_channels"]["P4"]
p4_channel_index = processed_OPM.ch_names.index(p4_channel_name)
p4_preprocessed_data = processed_OPM.get_data()[p4_channel_index]

In [28]:
times = processed_OPM.times

In [29]:
import matplotlib as mpl
from matplotlib.patches import Rectangle

def force_box(ax, lw=0.8, color="black"):
    # falls irgendwo axis('off') oder frame_off gesetzt wurde
    ax.set_axis_on()
    ax.set_frame_on(True)
    # Spines erzwingen
    for side in ("top","right","bottom","left"):
        sp = ax.spines[side]
        sp.set_visible(True)
        sp.set_linewidth(lw)
        sp.set_color(color)
        sp.set_zorder(10)  # sicherheitshalber nach vorne
    # Falls trotzdem unsichtbar (exotische Styles): Border als Patch
    # (Axes-Koordinaten 0..1)
    ax.add_patch(Rectangle((0,0), 1, 1, transform=ax.transAxes,
                           fill=False, lw=lw, edgecolor=color, zorder=9))

In [32]:
# === 2) Auswahl: Datei, Limb, Channels ===
file_id = "sub-91_EmgAcc_setupB_Move_processed"
limb    = "armL"
channels = ["brachioradialis_L_tkeo", "deltoideus_L_tkeo", "SVM_L"] #"brachioradialis_L_tkeo", ]

chan_data = {}
for ch in channels:
    on_s, off_s = get_events(events_result, file_id, limb, ch)
    thr         = get_threshold(thresholds_result, file_id, limb, ch)
    chan_data[ch] = {"on": on_s, "off": off_s, "thr": thr}

In [33]:
onsets_det = chan_data["deltoideus_L_tkeo"]["on"]

In [34]:
saving_path = find_path.get_onedrive_path(folder="figures")
saving_path = f"{saving_path}/mix_EMG_ACC_figures/sub-91_B_Move_OPM_with_legend.svg"

project folder found: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG


In [35]:
def shade_intervals(ax, on_list, off_list, color,alpha=0.2):
    for s, e in zip(on_list, off_list):
        if e is not None and s is not None:
            ax.axvspan(s, e, alpha=alpha, color=color, edgecolor="none", linewidth=0, antialiased=False)
            
            
def build_rest_intervals(on, off, left_edge=None, right_edge=None):
    """
    Liefert Start/Ende-Listen für 'Rest' = Lücken zwischen Bewegung (on..off),
    inkl. optional Bereich vor dem ersten on und nach dem letzten off.
    Erwartet on/off zeitlich sortiert und gepaart (on[i] < off[i]).
    """
    on = [t for t in on if t is not None]
    off = [t for t in off if t is not None]
    on.sort(); off.sort()

    # Falls der erste 'off' vor dem ersten 'on' liegt (oder counts ungleich): anpassen
    # (typisch, wenn die Sequenz im Rest startet)
    while off and on and off[0] < on[0]:
        off.pop(0)
    # gleiche Länge erzwingen
    m = min(len(on), len(off))
    on, off = on[:m], off[:m]

    rest_starts, rest_ends = [], []

    # Bereich vor dem ersten 'on'
    if left_edge is not None and on:
        if left_edge < on[0]:
            rest_starts.append(left_edge)
            rest_ends.append(on[0])

    # Lücken zwischen off[i] und on[i+1]
    for i in range(len(off) - 1):
        s = off[i]
        e = on[i + 1]
        if e > s:
            rest_starts.append(s)
            rest_ends.append(e)

    # Bereich nach dem letzten 'off'
    if right_edge is not None and off:
        if right_edge > off[-1]:
            rest_starts.append(off[-1])
            rest_ends.append(right_edge)

    return rest_starts, rest_ends

In [ ]:
# MEG plot # 
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12, 8), sharex=True, constrained_layout=False, gridspec_kw={'hspace': 0})
#plt.suptitle("Move - Setup B - full Setup synchronized") 
x_min, x_max = 15,30

# ---- Subplot 1: SVM_L ----
ax = axes[0]
ax.set_facecolor("white")
ax.grid(False)

ax.plot(df_move["Sync_Time (s)"], df_move["SVM_L"], color="steelblue", label="ACC SVM")
ax.set_xlim(x_min, x_max)
ax.set_ylim(-0.05, 0.5)
#shade_intervals(ax, chan_data["SVM_L"]["on"], chan_data["SVM_L"]["off"])
#ax.set_xlabel("Time (s)")
#ax.set_title("acceleration")
ax.set_yticks([0, 0.25, 0.5])
for side in (["left", "right", "top", "bottom"]):
    sp = ax.spines[side]
    sp.set_linewidth(0.2)
    sp.set_color("black")
    sp.set_zorder(10)
ax.tick_params(bottom=True, left=True)
ax.legend(bbox_to_anchor=(1.02,0.5), loc="center left", frameon=False)

# ---- Subplot 2: main plot mit MEG daten! ----
ax = axes[1]
ax.set_facecolor("white")
ax.grid(False)

ax.plot(times, f4_preprocessed_data, color="red", label="F4")
ax.plot(times, p4_preprocessed_data, color="orange", label="P4")
ax.plot(times, c4_preprocessed_data, color="pink", label="C4")
for x in chan_data["deltoideus_L_tkeo"]["on"]:
    if x > 29.1:
        continue
    else:
        ax.axvline(x, color="gray", linestyle="--", label="EMG onset delt") 
for x in chan_data["deltoideus_L_tkeo"]["off"]:
    if x > 29.5:
        continue
    else:
        ax.axvline(x, color="silver", linestyle="--", label="EMG offset delt") 

ons = [t for t in chan_data["SVM_L"]["on"] if x_min <= t <= x_max]
offs = [t for t in chan_data["SVM_L"]["off"] if x_min <= t <= x_max]
    
shade_intervals(ax, chan_data["SVM_L"]["on"], chan_data["SVM_L"]["off"], color="darkseagreen", alpha=0.2)
rest_starts, rest_ends = build_rest_intervals(ons, offs, left_edge=x_min, right_edge=x_max)
shade_intervals(ax, rest_starts, rest_ends, color="lightsteelblue", alpha=0.2)

ax.set_xlim(x_min, x_max)
for side in (["left", "right"]):
    sp = ax.spines[side]
    sp.set_linewidth(0.2)
    sp.set_color("black")
    sp.set_zorder(10)

ax.tick_params(left=True)
ax.set_yticks([-3*1e-12, 0, 3*1e-12])
ax.legend(bbox_to_anchor=(1.02,0.5), loc="center left", frameon=False)

# ---- Subplot 3: beide EMGs hier rein? also delt und brachio? ----
ax = axes[2]
ax.set_facecolor("white")
ax.grid(False)

ax.plot(df_move["Sync_Time (s)"], df_move["deltoideus_L"], label="deltoideus left", color="cornflowerblue")
ax.plot(df_move["Sync_Time (s)"], df_move["brachioradialis_L"], label="brachioradialis left", color="lightskyblue")
#for x in chan_data["deltoideus_L_tkeo"]["on"]:
#    ax.axvline(x, color="gray", linestyle="--")
#for x in chan_data["deltoideus_L_tkeo"]["off"]:
#    ax.axvline(x, color="silver", linestyle="--") 
#shade_intervals(ax, chan_data["SVM_L"]["on"], chan_data["SVM_L"]["off"])
ax.set_xlim(x_min, x_max)
ax.set_yticks([-1500, 0, 1500])
ax.set_xticks([16, 18, 20, 22, 24, 26, 28, 30])
ax.tick_params(left=True, bottom=True)
#ax.set_title("EMG data")
for side in (["left", "right", "bottom", "top"]):
    sp = ax.spines[side]
    sp.set_linewidth(0.2)
    sp.set_color("black")
    sp.set_zorder(10) 
    

ax.legend(bbox_to_anchor=(1.02,0.5), loc="center left", frameon=False)
plt.tight_layout()
plt.show()
#fig.savefig(saving_path, dpi=300, bbox_inches="tight", pad_inches=0.05)

### Threshold inset plot ###

In [360]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np

def add_x_zoom_inset(
    ax, x0, x1, *,
    thr=None, thr_style=None,
    # Neue Optionen für Y:
    ylims=None,            # tuple (ymin, ymax) -> explizit setzen
    y_center=None,         # z.B. thr
    y_span=None,           # z.B. 0.2 -> zeigt [y_center - y_span/2, y_center + y_span/2]
    y_pad=0.05,            # relativer Rand, wenn auto
    loc="upper right", width="45%", height="38%", borderpad=0.6,
    show_connectors=True, hide_inset_ticks=True
):
    axins = inset_axes(ax, width=width, height=height, loc=loc, borderpad=borderpad)

    # Linien/Daten aus Haupt-Plot neu zeichnen
    for line in ax.get_lines():
        axins.plot(line.get_xdata(), line.get_ydata(),
                   color=line.get_color(),
                   linestyle=line.get_linestyle(),
                   linewidth=line.get_linewidth(),
                   alpha=(1.0 if line.get_alpha() is None else line.get_alpha()))

    # X-Zoom
    axins.set_xlim(x0, x1)

    # --- Y-Skalierung ---
    if ylims is not None:
        y_min, y_max = ylims
    elif (y_center is not None) and (y_span is not None):
        y_min = y_center - y_span/2.0
        y_max = y_center + y_span/2.0
    else:
        # automatisch aus Daten im Zoombereich (inkl. Schwelle)
        ys = []
        for line in ax.get_lines():
            x = np.asarray(line.get_xdata()); y = np.asarray(line.get_ydata())
            m = (x >= x0) & (x <= x1)
            if m.any(): ys.append(y[m])
        if ys:
            y_all = np.concatenate(ys)
            y_min, y_max = float(np.min(y_all)), float(np.max(y_all))
        else:
            y_min, y_max = ax.get_ylim()

        # Schwelle berücksichtigen
        if thr is not None:
            tvals = (thr if np.iterable(thr) and not isinstance(thr, (str, bytes)) else [thr])
            y_min = min([y_min] + list(tvals))
            y_max = max([y_max] + list(tvals))

        pad = y_pad * (y_max - y_min if y_max > y_min else 1.0)
        y_min -= pad; y_max += pad

    # Falls identische Grenzen: minimalen Spread erzwingen
    if y_min == y_max:
        eps = 1e-6 * (abs(y_min) + 1.0)
        y_min -= eps; y_max += eps

    axins.set_ylim(y_min, y_max)

    # Schwelle(n) im Inset
    if thr is not None:
        style = dict(color="0.25", linestyle="--", linewidth=1, zorder=5)
        if thr_style: style.update(thr_style)
        try:
            for t in thr: axins.axhline(t, **style)
        except TypeError:
            axins.axhline(thr, **style)

    # Inset aufräumen
    if hide_inset_ticks:
        axins.tick_params(axis='both', which='both', length=0)
        axins.set_xticklabels([]); axins.set_yticklabels([])

    if show_connectors:
        ax.figure.canvas.draw()
        ax.indicate_inset_zoom(axins, edgecolor="0.3")

    return axins

def draw_onoff_inset(axins, on, off, *,
                     on_style=None, off_style=None, zorder=6):
    on_style  = {'color': 'gray',   'linestyle': '--', 'linewidth': 1, **(on_style  or {})}
    off_style = {'color': 'silver', 'linestyle': '--', 'linewidth': 1, **(off_style or {})}
    x0, x1 = axins.get_xlim()
    for x in on:
        if x0 <= x <= x1:
            axins.axvline(x, zorder=zorder, **on_style)
    for x in off:
        if x0 <= x <= x1:
            axins.axvline(x, zorder=zorder, **off_style)

In [370]:
move_thresh_path = f"{fig_path}/mix_EMG_ACC_figures/sub-91_B_Move_armL_data_withActive_andThresh_legend.svg"

In [375]:
# arme!
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12, 8), sharex=True)
x_min, x_max = 20, 25

# ---- Subplot 1: brachioradialis_L_tkeo ----
ax = axes[0]
ax.plot(df_move["Sync_Time (s)"], df_move["brachioradialis_L_tkeo"], color="lightskyblue", label="brachioradialis")
#shade_intervals(ax, chan_data["brachioradialis_L_tkeo"]["on"], chan_data["brachioradialis_L_tkeo"]["off"])

thresh = chan_data["brachioradialis_L_tkeo"]["thr"]
on, off = chan_data["brachioradialis_L_tkeo"]["on"], chan_data["brachioradialis_L_tkeo"]["off"]
for x in on:
    ax.axvline(x, color="gray", linestyle="--")
for x in off:
    ax.axvline(x, color="silver", linestyle="--")

draw_threshold(ax, thresh)
ax.set_facecolor("white")
ax.set_xlim(x_min, x_max)
ax.set_ylim(-1, 0.15*1e6)
#ax.set_title("brachioradialis_L")
ax.grid(False)

ax.tick_params(left=True)
ax.set_yticks([0, 0.075*1e6, 0.15*1e6])

sp = ax.spines["left"]
sp.set_linewidth(0.5)
sp.set_color("black")
sp.set_zorder(10)

axins = add_x_zoom_inset(ax, 22.9, 23.9, loc="upper right", thr=thresh, ylims=(4, 2000),  width="42%", height="50%")
draw_onoff_inset(axins,
                 chan_data["brachioradialis_L_tkeo"]["on"],
                 chan_data["brachioradialis_L_tkeo"]["off"])

# ---- Subplot 2: deltoideus_L_tkeo ----
ax = axes[1]
ax.plot(df_move["Sync_Time (s)"], df_move["deltoideus_L_tkeo"], color="cornflowerblue", label="deltoideus")
#shade_intervals(ax, chan_data["deltoideus_L_tkeo"]["on"], chan_data["deltoideus_L_tkeo"]["off"])
thresh = chan_data["deltoideus_L_tkeo"]["thr"]
draw_threshold(ax, thresh)
on, off = chan_data["deltoideus_L_tkeo"]["on"], chan_data["deltoideus_L_tkeo"]["off"]
for x in on:
    ax.axvline(x, color="gray", linestyle="--")
for x in off:
    ax.axvline(x, color="silver", linestyle="--")
ax.set_facecolor("white")
ax.set_xlim(x_min, x_max)
ax.set_ylim(0, 1*1e6)
#ax.set_title("deltoideus_L")
ax.grid(False)
sp = ax.spines["left"]
sp.set_linewidth(0.5)
sp.set_color("black")
sp.set_zorder(10)

ax.tick_params(left=True)
ax.set_yticks([0, 0.5*1e6, 1*1e6])

axins = add_x_zoom_inset(ax, 23.2, 24.2, loc="upper right", thr=thresh, ylims=(2, 6000), width="42%", height="50%")
draw_onoff_inset(axins,
                 chan_data["deltoideus_L_tkeo"]["on"],
                 chan_data["deltoideus_L_tkeo"]["off"])

# ---- Subplot 3: SVM_L ----
ax = axes[2]
ax.plot(df_move["Sync_Time (s)"], df_move["SVM_L"], color="steelblue", label="ACC SVM")
#shade_intervals(ax, chan_data["SVM_L"]["on"], chan_data["SVM_L"]["off"])
draw_threshold(ax, chan_data["SVM_L"]["thr"])
on, off = chan_data["SVM_L"]["on"], chan_data["SVM_L"]["off"]
for x in on:
    ax.axvline(x, color="gray", linestyle="--")
for x in off:
    ax.axvline(x, color="silver", linestyle="--")
ax.set_facecolor("white")
ax.set_xlim(x_min, x_max)
ax.set_ylim(0, 1)
#ax.set_title("SVM_L")
ax.set_xlabel("Time (s)")
ax.grid(False)
ax.tick_params(bottom=True, left=True)
ax.set_yticks([0, 0.5, 1])

#ax.spines[['top', 'left', "bottom"]].set_visible(False)
for side in (["bottom", "left"]):
    sp = ax.spines[side]
    sp.set_linewidth(0.5)
    sp.set_color("black")
    sp.set_zorder(10)
#ax.set_xticks([20,30,40,50,60,70,80,90])

axins = add_x_zoom_inset(ax, 23.8, 24.8, loc="upper right", thr=chan_data["SVM_L"]["thr"], ylims=(0, 0.25), width="42%", height="50%")


ax.legend(loc="upper left", frameon=False)
plt.tight_layout()
#plt.show()
fig.savefig(move_thresh_path, dpi=300)

C:\Users\User\AppData\Local\Temp\ipykernel_1692\3875586171.py:97: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


donut plot

In [366]:
subjects = ["sub-E", "sub-F", "sub-L", "sub-P"]
test_counts = np.array([2043, 2120, 2062, 2068], dtype=int)
total = int(8293)  # konstant; alternativ: total = int(test_counts.sum())
train_counts = total - test_counts  # Train-Size je Fold (LOSO)

# --- Figure mit 2 Panels: Donut (links), Stacked Bars (rechts) ---
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(11, 4.8), constrained_layout=True)

# 1) Donut: Anteil der Subjekte am Gesamtdatensatz
colors = ["#4e79a7", "#f28e2b", "#e15759", "#76b7b2"]
wedges, _ = ax0.pie(test_counts, startangle=90, colors=colors, wedgeprops=dict(width=0.38))
ax0.set_aspect('equal')
ax0.set_title("Subject share of total windows", pad=8)

# Beschriftung mit Prozenten + N
pct = 100 * test_counts / test_counts.sum()
labels = [f"{s}\n{n} ({p:.1f}%)" for s, n, p in zip(subjects, test_counts, pct)]
# Legende statt direkt auf den Pie schreiben (besser in der BA)
ax0.legend(wedges, labels, loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)

# 2) Stacked Bars: Train vs. Test je Fold (Subjekt als Test)
x = np.arange(len(subjects))
bar_w = 0.55
ax1.bar(x, train_counts, width=bar_w, label="Train (others)", color="#c7dceb")
ax1.bar(x, test_counts,  width=bar_w, bottom=train_counts, label="Test (held-out subj.)", color="#4e79a7")
ax1.set_xticks(x)
ax1.set_xticklabels(subjects)
ax1.set_ylabel("Windows")
ax1.set_title("LOSO fold sizes per test subject", pad=8)

# Mittelwert-Linie für Testgröße (zeigt Balance)
mean_test = test_counts.mean()
ax1.axhline(mean_test, color="0.3", linestyle="--", linewidth=1)
ax1.text(x[-1]+0.4, mean_test, f"mean test ≈ {mean_test:.0f}", va="center", ha="left", color="0.3")

# Feintuning
for ax in (ax0, ax1):
    ax.set_facecolor("white")
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.legend(frameon=False, loc="upper right")

plt.show()
# fig.savefig("loso_balance.pdf") 